
CBOW


In [1]:
import pandas as pd 

artigo_treino = pd.read_csv("./treino.csv")
artigo_teste = pd.read_csv("./teste.csv")

In [2]:
from sklearn.feature_extraction.text import CountVectorizer

texto = [
    "Este produto é muito bom",
    "Este produto é muito ruim"
]

vetorizador = CountVectorizer()
vetorizador.fit(texto)
print(vetorizador.vocabulary_)

{'este': 1, 'produto': 3, 'muito': 2, 'bom': 0, 'ruim': 4}


In [3]:
# from huggingface_hub import hf_hub_download
# from safetensors.numpy import load_file

# path = hf_hub_download(repo_id="nilc-nlp/word2vec-cbow-300d",
#                        filename="embeddings.safetensors")

# data = load_file(path)
# vectors = data["embeddings"]

# vocab_path = hf_hub_download(repo_id="nilc-nlp/word2vec-cbow-300d",
#                              filename="vocab.txt")
# with open(vocab_path) as f:
#     vocab = [w.strip() for w in f]

# print(vectors.shape)


In [4]:
from huggingface_hub import hf_hub_download
from safetensors.numpy import load_file
from gensim.models import KeyedVectors

# embeddings
path = hf_hub_download(
    repo_id="nilc-nlp/word2vec-cbow-300d",
    filename="embeddings.safetensors"
)
data = load_file(path)
vectors = data["embeddings"]

# vocabulário
vocab_path = hf_hub_download(
    repo_id="nilc-nlp/word2vec-cbow-300d",
    filename="vocab.txt"
)
with open(vocab_path, encoding="utf-8") as f:
    vocab = [w.strip() for w in f]

# checagem de sanidade
print(len(vocab), vectors.shape)

# Word2Vec no gensim
modelo = KeyedVectors(vector_size=vectors.shape[1])
modelo.add_vectors(vocab, vectors)


929606 (929606, 300)


In [5]:
data.keys()


dict_keys(['embeddings'])

In [6]:
modelo.most_similar("amor")


[('afeto', 0.7056611180305481),
 ('tormento', 0.6202821135520935),
 ('desamor', 0.6199557781219482),
 ('ciúme', 0.6167261600494385),
 ('arrependimento', 0.6010848879814148),
 ('encanto', 0.5992918014526367),
 ('instincto', 0.5957212448120117),
 ('ódio', 0.580238401889801),
 ('infortúnio', 0.5744850039482117),
 ('egoísmo', 0.5689378380775452)]

In [7]:
modelo.get_vector("china")

array([-1.49033e-01,  1.26020e-01,  2.17628e-01,  1.82684e-01,
        1.65151e-01, -1.59660e-01, -2.34411e-01,  6.00570e-02,
        8.03680e-02,  2.87578e-01, -4.81100e-03, -5.68800e-02,
        2.15676e-01,  8.65540e-02,  1.25983e-01,  3.36157e-01,
       -1.83254e-01, -1.18499e-01,  1.13010e-02,  1.03814e-01,
        9.37640e-02,  2.90178e-01, -1.64395e-01, -1.13300e-02,
       -1.80676e-01, -1.15820e-02,  1.08728e-01,  1.65898e-01,
        9.37900e-02,  2.66767e-01, -1.29890e-02,  9.16030e-02,
        2.21292e-01, -1.36497e-01, -4.26350e-02, -1.30038e-01,
        2.17067e-01, -1.01963e-01, -3.70960e-02,  1.42155e-01,
        3.41109e-01,  2.46560e-01,  1.27458e-01,  5.72360e-02,
       -1.47962e-01, -1.60290e-02,  1.86533e-01,  7.71550e-02,
       -3.50024e-01, -4.06085e-01,  1.67131e-01, -4.75230e-02,
        5.13780e-02, -1.28224e-01,  1.06580e-02, -2.92652e-01,
        1.40540e-01, -4.57049e-01,  1.31094e-01,  2.03234e-01,
        2.94019e-01,  7.38370e-02,  1.11554e-01, -1.642

In [8]:
modelo.most_similar("china")

[('rússia', 0.7320705056190491),
 ('índia', 0.7241616249084473),
 ('tailândia', 0.701935887336731),
 ('indonésia', 0.6860769987106323),
 ('turquia', 0.6741335988044739),
 ('malásia', 0.6665689945220947),
 ('mongólia', 0.6593616008758545),
 ('manchúria', 0.658184826374054),
 ('urss', 0.6581669449806213),
 ('grã-bretanha', 0.6568097472190857)]

In [9]:
modelo.most_similar(positive= ["china","brasil"])

[('japão', 0.725788950920105),
 ('ocidente', 0.5933547019958496),
 ('canadá', 0.5921916961669922),
 ('austrália', 0.5688517093658447),
 ('tibete', 0.5675076246261597),
 ('haiti', 0.5656365752220154),
 ('irã', 0.5653263330459595),
 ('camboja', 0.5621733069419861),
 ('índia', 0.5570496916770935),
 ('rússia', 0.5542328953742981)]

In [12]:
modelo.most_similar(positive= ["carros","foguete"], negative="carro")

[('foguetes', 0.5930031538009644),
 ('foguetões', 0.5570027828216553),
 ('caças', 0.550533652305603),
 ('aviões', 0.5263278484344482),
 ('satélites', 0.5231359601020813),
 ('motores', 0.51063472032547),
 ('hidroaviões', 0.4993920922279358),
 ('vaivéns', 0.48650243878364563),
 ('veículos', 0.4795992970466614),
 ('barcos', 0.47358769178390503)]